In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import locale 
import matplotlib.dates as mdates
from datetime import datetime

from pathlib import Path 

In [2]:
YEAR_MONTH_SORT = {
    'by': ['Leto', 'Mesec'],
    'key': lambda col: col if col.name != 'Mesec' else col.map(lambda x: datetime.strptime(x, '%b').month)
}

In [ ]:
locale.setlocale(locale.LC_ALL, "sl_SI.utf8")

regije_filter = {
    "SO2": [
        "Zasavska",
        "Savinjska",
        "Posavska",
        "Pomurska",
        "Podravska",
        "Osrednjeslovenska",

    ]
}

def filter_and_aggreg(df: pd.DataFrame, years=None, regions_filter=[])->pd.DataFrame:
    df = df[df['Regija'].isin(regions_filter)]

    display(df['Regija'].unique())




for csv_file in sorted(list(Path("podatki/df_regije").iterdir()), reverse=True):
    snov = csv_file.stem

    raw_df = pd.read_csv(csv_file)
    
    prepare_df(df=raw_df,regions_filter=regije_filter[snov])
    break 
    
    df = regije_dict[name].copy()
    df.dropna(inplace=True)
    df.drop(columns="Postaja", inplace=True)

    df_reg = df.groupby(["Regija", "Leto", "Mesec"])[name].agg("mean").reset_index()

    df_reg.sort_values(**YEAR_MONTH_SORT, inplace=True)

    df_reg.loc[:, "Date"] = pd.to_datetime(
        df_reg["Leto"].astype(str) + "-" + df_reg["Mesec"], format="%Y-%b"
    ) + pd.offsets.MonthEnd(0)

    regije = regije_filter[name]

    df_reg = df_reg[df_reg["Regija"].isin(regije)]

    sns.set_style("whitegrid")
    plt.figure(figsize=(10, 6))

    sns.lineplot(data=df_reg, x="Date", y=name, hue="Regija", linewidth=1)

    years = df_reg["Date"].dt.year.unique()

    ax = plt.gca()

    odd_years = years[years % 2 != 0]
    ax.set_xticks([pd.to_datetime(f"{year}-01-01") for year in odd_years])
    ax.set_xticklabels(odd_years)

    necd_year = pd.to_datetime("2005-01-01")
    plt.axvline(
        x=necd_year, color="red", linestyle="-", linewidth=1.5, alpha=0.5
    )  # Add transparency and remove arrow
    plt.annotate(
        "NECD direktiva postane pravnomočna",
        xy=(necd_year, max(df_reg[name])),
        xytext=(
            necd_year + pd.Timedelta(days=125),
            max(df[name]) * 0.75,
        ),  # Adjust x-position of text
        horizontalalignment="left",
        verticalalignment="top",
        bbox=dict(
            facecolor="red",
            alpha=0.5,
        ),
        color="white",
    )  # Add transparent red box

    plt.xticks(rotation=45)
    plt.xlabel("Leto")
    plt.ylabel(f"nivo {name} [µg/m3]")
    plt.title(f"Trend gibanja {name} po regijah")

    plt.legend(title="Regija", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(f"grafi/trend_gibanja_{name}_regije.png")
    plt.show()

    break

array(['Osrednjeslovenska', 'Podravska', 'Savinjska', 'Zasavska',
       'Posavska', 'Pomurska', 'Goriška'], dtype=object)